In [23]:
import pandas as pd
import numpy as np
import sys
import os

In [24]:
sys.path.append(os.path.join(os.getcwd(), 'src'))

In [25]:
def calculate_ahp_weights(pairwise_matrix):
    """
    Tính toán trọng số AHP và Tỷ số Nhất quán (CR)
    """
    n = pairwise_matrix.shape[0]
    
    try:
        col_sums = pairwise_matrix.sum(axis=0)
        col_sums[col_sums == 0] = 1e-9
        norm_matrix = pairwise_matrix / col_sums
        weights = norm_matrix.mean(axis=1)
    except Exception as e:
        print(f"Lỗi khi chuẩn hóa ma trận: {e}")
        return None, None, "Lỗi khi chuẩn hóa ma trận"

    RI_lookup = {
        1: 0.00, 2: 0.00, 3: 0.58, 4: 0.90, 5: 1.12, 
        6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49, 
        11: 1.51
    }

    try:
        A_x = pairwise_matrix.dot(weights)
        weights_safe = np.where(weights == 0, 1e-9, weights)
        lambda_max = (A_x / weights_safe).mean()
        CI = (lambda_max - n) / (n - 1) if n > 1 else 0
        RI = RI_lookup.get(n, 1.59)
        CR = CI / RI if RI != 0 else 0
    except Exception as e:
        return weights, None, f"Lỗi khi tính toán CR: {e}"

    return weights, CR, None

In [26]:
def run_topsis_analysis(decision_matrix, ahp_weights, criteria_types):
    """
    Thực hiện phân tích TOPSIS và trả về điểm số.
    """
    matrix = decision_matrix.values.astype(float)
    
    try:
        norm_denominator = np.linalg.norm(matrix, axis=0)
        norm_denominator[norm_denominator == 0] = 1e-9
        norm_matrix = matrix / norm_denominator
    except Exception as e:
        print(f"Lỗi khi chuẩn hóa ma trận TOPSIS: {e}")
        return None

    weighted_matrix = norm_matrix * ahp_weights

    ideal_best = np.zeros(matrix.shape[1])
    ideal_worst = np.zeros(matrix.shape[1])

    for j in range(matrix.shape[1]):
        if criteria_types[j] == 'benefit':
            ideal_best[j] = np.max(weighted_matrix[:, j])
            ideal_worst[j] = np.min(weighted_matrix[:, j])
        elif criteria_types[j] == 'cost':
            ideal_best[j] = np.min(weighted_matrix[:, j])
            ideal_worst[j] = np.max(weighted_matrix[:, j])

    dist_best = np.linalg.norm(weighted_matrix - ideal_best, axis=1)
    dist_worst = np.linalg.norm(weighted_matrix - ideal_worst, axis=1)

    epsilon = 1e-9
    closeness_score = dist_worst / (dist_best + dist_worst + epsilon)

    return closeness_score

In [27]:
# Load dữ liệu
df_decision = pd.read_csv('../src/Data Preprocessing/DECISION_MATRIX_FOR_TOPSIS.csv')
print("Ma trận quyết định:")
display(df_decision)
CRITERIA_TYPES = [
    'cost', 'cost', 'benefit',  # Định giá
    'benefit', 'benefit', 'benefit',  # Sinh lời
    'cost', 'cost', 'cost',  # Rủi ro
    'benefit', 'cost'  # Hiệu quả
]

ALL_CRITERIA_ORDERED = ['P/E (TTM)', 'P/B', 'EPS (TTM)', 'ROE', 'ROA', 'NIM', 'D_E', 'LDR', 'NPL_Ratio', 'Asset_Turnover', 'CIR']

Ma trận quyết định:


,ticker,P/E (TTM),P/B,EPS (TTM),ROE,ROA,D_E,LDR,NPL_Ratio,Asset_Turnover,NIM,CIR
0,VCB.VN,22.498590,2.232644,2644.61,20.11,1.52,12.202737,0.846234,0.64,0.054072,9.00,64.84
1,BID.VN,13.332171,1.652211,2869.00,12.56,0.62,19.406761,0.974815,1.01,0.053206,11.55,85.55
2,CTG.VN,7.925258,1.565975,6208.00,15.18,0.93,15.354462,0.973163,1.26,0.054418,10.70,78.90
3,OCB.VN,8.481765,1.011455,1461.96,20.20,2.39,7.460947,1.032854,1.32,0.076545,7.04,60.92


In [28]:
# Kịch bản 1: Ưu tiên cân bằng (Equal weights)
print("\n=== KỊCH BẢN 1: Ưu tiên cân bằng ===")
equal_weights = np.ones(len(ALL_CRITERIA_ORDERED)) / len(ALL_CRITERIA_ORDERED)
scores_1 = run_topsis_analysis(df_decision[ALL_CRITERIA_ORDERED], equal_weights, CRITERIA_TYPES)
df_results_1 = pd.DataFrame({'Ticker': df_decision['ticker'], 'TOPSIS_Score': scores_1})
df_results_1['Rank'] = df_results_1['TOPSIS_Score'].rank(ascending=False).astype(int)
df_results_1 = df_results_1.sort_values(by='Rank')
display(df_results_1)


=== KỊCH BẢN 1: Ưu tiên cân bằng ===


,Ticker,TOPSIS_Score,Rank
3,OCB.VN,0.571184,1
2,CTG.VN,0.557374,2
0,VCB.VN,0.404361,3
1,BID.VN,0.338031,4


In [29]:
# Kịch bản 2: Ưu tiên mạnh về Sinh lời (Profitability)
print("\n=== KỊCH BẢN 2: Ưu tiên mạnh về Sinh lời ===")
group_weights = np.array([0.1, 0.6, 0.2, 0.1])  # Định giá, Sinh lời, Rủi ro, Hiệu quả
local_weights = {
    'Định giá': np.array([0.33, 0.33, 0.34]),
    'Khả năng sinh lời': np.array([0.5, 0.3, 0.2]),
    'Sức khỏe tài chính': np.array([0.33, 0.33, 0.34]),
    'Hiệu quả hoạt động': np.array([0.5, 0.5])
}
global_weights_2 = np.concatenate([
    group_weights[0] * local_weights['Định giá'],
    group_weights[1] * local_weights['Khả năng sinh lời'],
    group_weights[2] * local_weights['Sức khỏe tài chính'],
    group_weights[3] * local_weights['Hiệu quả hoạt động']
])
scores_2 = run_topsis_analysis(df_decision[ALL_CRITERIA_ORDERED], global_weights_2, CRITERIA_TYPES)
df_results_2 = pd.DataFrame({'Ticker': df_decision['ticker'], 'TOPSIS_Score': scores_2})
df_results_2['Rank'] = df_results_2['TOPSIS_Score'].rank(ascending=False).astype(int)
df_results_2 = df_results_2.sort_values(by='Rank')
display(df_results_2)


=== KỊCH BẢN 2: Ưu tiên mạnh về Sinh lời ===


,Ticker,TOPSIS_Score,Rank
3,OCB.VN,0.756230,1
0,VCB.VN,0.593702,2
2,CTG.VN,0.319243,3
1,BID.VN,0.200945,4


In [30]:

# Kịch bản 3: Ưu tiên mạnh về Rủi ro thấp (Low Risk)
print("\n=== KỊCH BẢN 3: Ưu tiên mạnh về Rủi ro thấp ===")
# Giả lập trọng số AHP: Nhóm Rủi ro có trọng số cao
group_weights_3 = np.array([0.1, 0.1, 0.6, 0.2])  # Định giá, Sinh lời, Rủi ro, Hiệu quả
global_weights_3 = np.concatenate([
    group_weights_3[0] * local_weights['Định giá'],
    group_weights_3[1] * local_weights['Khả năng sinh lời'],
    group_weights_3[2] * local_weights['Sức khỏe tài chính'],
    group_weights_3[3] * local_weights['Hiệu quả hoạt động']
])
scores_3 = run_topsis_analysis(df_decision[ALL_CRITERIA_ORDERED], global_weights_3, CRITERIA_TYPES)
df_results_3 = pd.DataFrame({'Ticker': df_decision['ticker'], 'TOPSIS_Score': scores_3})
df_results_3['Rank'] = df_results_3['TOPSIS_Score'].rank(ascending=False).astype(int)
df_results_3 = df_results_3.sort_values(by='Rank')
display(df_results_3)



=== KỊCH BẢN 3: Ưu tiên mạnh về Rủi ro thấp ===


,Ticker,TOPSIS_Score,Rank
0,VCB.VN,0.645970,1
3,OCB.VN,0.565997,2
2,CTG.VN,0.326049,3
1,BID.VN,0.251314,4


In [31]:
print("\n=== SO SÁNH CÁC KỊCH BẢN ===")

# 1. Tạo các Series (ánh xạ) từ Ticker -> Rank cho mỗi kịch bản
rank_1_map = df_results_1.set_index('Ticker')['Rank']
rank_2_map = df_results_2.set_index('Ticker')['Rank']
rank_3_map = df_results_3.set_index('Ticker')['Rank']

# 2. Tạo DataFrame so sánh
comparison = pd.DataFrame({
    'Ticker': df_decision['ticker'],
    'Rank_KB1': df_decision['ticker'].map(rank_1_map),
    'Rank_KB2': df_decision['ticker'].map(rank_2_map),
    'Rank_KB3': df_decision['ticker'].map(rank_3_map)
})

display(comparison)


=== SO SÁNH CÁC KỊCH BẢN ===


,Ticker,Rank_KB1,Rank_KB2,Rank_KB3
0,VCB.VN,3,2,1
1,BID.VN,4,4,4
2,CTG.VN,2,3,3
3,OCB.VN,1,1,2
